In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.animation import FuncAnimation, FFMpegWriter
import os
from matplotlib.gridspec import GridSpec
import subprocess
import multiprocessing

In [2]:

# === Setup constants ===
SolarConstant = 1367
LATS = np.linspace(-90, 90, 180)
LONS = np.linspace(-180, 180, 360)
lon_grid, lat_grid = np.meshgrid(LONS, LATS)
lat_rad = np.radians(lat_grid)
events = {
    79:  "Vernal Equinox",
    172: "Summer Solstice",
    266: "Autumnal Equinox",
    355: "Winter Solstice"
}

MONTH_DAYS = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
MONTH_NAMES = ["Jan", "Feb", "Mar", "Apr", "May", "Jun",
               "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

# === Solar and geometry functions ===
def solar_declination(day):
    return np.radians(23.44) * np.sin(2 * np.pi * (day - 81) / 365.0)

def calculate_zenith_cosine(decl, utc_hour):
    lst = utc_hour + lon_grid / 15.0
    hra = np.radians(15 * (lst - 12))
    cos_zenith = np.sin(lat_rad) * np.sin(decl) + \
                 np.cos(lat_rad) * np.cos(decl) * np.cos(hra)
    return np.clip(cos_zenith, 0, 1)

def get_subsolar_point(day, hour):
    decl = solar_declination(day)
    lat = np.degrees(decl)
    lon = -15 * (hour - 12)
    return lat, lon, decl

titles_text = ["North Pole (AE)", "South Pole (AE)", "Global"]

# === Main function to render monthly frames ===
def render_month(month_index,):
    start_day = sum(MONTH_DAYS[:month_index]) + 1
    end_day = start_day + MONTH_DAYS[month_index] - 1
    output_dir = f"frames_{MONTH_NAMES[month_index]}"
    os.makedirs(output_dir, exist_ok=True)

    for day in range(start_day, end_day + 1):
        for hour in range(0, 24, 3):
            frame = (day - 1) * 24 + hour
            lat, lon, decl = get_subsolar_point(day, hour)
            cos_zenith = calculate_zenith_cosine(decl, hour)

            projections = [
                ccrs.AzimuthalEquidistant(central_longitude=0, central_latitude=90),
                ccrs.AzimuthalEquidistant(central_longitude=0, central_latitude=-90),
                ccrs.Mollweide(central_longitude= 0 )
            ]

            fig = plt.figure(figsize=(12, 8))
            gs = GridSpec(2, 2, height_ratios=[1.2, 1])

            # Top row: AE maps side by side
            ax1 = plt.subplot(gs[0, 0], projection=projections[0])
            ax2 = plt.subplot(gs[0, 1], projection=projections[1])

            # Bottom row: Mollweide spanning both columns
            ax3 = plt.subplot(gs[1, :], projection=projections[2])

            axes = [ax1, ax2, ax3]

            

            for i, ax in enumerate(axes):
                ax.set_global()
                ax.coastlines()
                ax.add_feature(cfeature.BORDERS, linewidth=0.5)
                ax.gridlines(draw_labels=False, linewidth=0.3, linestyle='--')
                mesh = ax.pcolormesh(
                    LONS, LATS, SolarConstant*cos_zenith, transform=ccrs.PlateCarree(),
                    cmap='inferno', shading='auto', vmin=0, vmax=SolarConstant
                )
                ax.plot(lon, lat, 'o', color='yellow', transform=ccrs.PlateCarree(), zorder=5)
                ax.set_title(f"{titles_text[i]}", fontsize=10)

            cb_ax = fig.add_axes([0.125, 0.05, 0.75, 0.03])
            cb = plt.colorbar(mesh, cax=cb_ax, orientation='horizontal')
            ticks = [0, 200, 400, 600, 800, 1000, 1200]
            cb.set_ticks(ticks)
            cb.set_label("Estimated Solar Irradiance (W/m$^{2}$)")
            
            if day in events:
                fig.suptitle(f"Sunlight Distribution on Day {day} — {hour:02d}:00 UTC\n" + events[day],
                    fontsize=14, ha='center', y=0.95)
            else:
                fig.suptitle(f"Sunlight Distribution on Day {day} — {hour:02d}:00 UTC",
                    fontsize=14, ha='center', y=0.95)
            
            filename = os.path.join(output_dir, f"frame_{day:03d}_{hour:02d}.png")
            fig.savefig(filename, dpi=100, bbox_inches='tight')
            plt.close(fig)
    print(f"{MONTH_NAMES[month_index]} Done!")


In [3]:
def main():
    with multiprocessing.Pool() as pool:
        pool.map(render_month, range(12))  # Renders months 0–11 in parallel

if __name__ == "__main__":
    main()


Feb Done!
Sep Done!
Jun Done!
Apr Done!
Nov Done!
May Done!
Oct Done!
Jan Done!
Mar Done!
Dec Done!
Jul Done!
Aug Done!


---

# Prepare for Combination

In [5]:
def generate_frame_list(month, events):

    month_dir = f"frames_{month}"
    frame_files = sorted(
        [f for f in os.listdir(month_dir) if f.endswith(".png")],
        key=lambda name: tuple(int(x) for x in name.replace(".png", "").split("_")[1:])
    )
    frame_list_path = f"{month_dir}/frame_list.txt"

    with open(frame_list_path, "w") as f:
        for i, frame in enumerate(frame_files):
            full_path = os.path.abspath(os.path.join(month_dir, frame))
            parts = frame.split("_")
            try:
                day = int(parts[1])
            except (IndexError, ValueError):
                day = None

            # Number of duplicates (1 means just original, >1 means duplicates)
            num_copies = 3 if day in events else 1
            duration = 0.075

            for copy_idx in range(num_copies):
                f.write(f"file '{full_path}'\n")

                # Write duration after every frame except the very last one overall
                # Calculate if this is the last file line overall
                is_last_file = (i == len(frame_files) - 1) and (copy_idx == num_copies - 1)
                if not is_last_file:
                    f.write(f"duration {duration}\n")

    return frame_list_path

In [6]:
for month in MONTH_NAMES:
        print(f"Generating video for {month}")
        frame_list = generate_frame_list(month, events)
        cmd = [
            "ffmpeg", "-y", "-f", "concat", "-safe", "0",
            "-i", frame_list,
            "-fps_mode", "vfr",
            "-vf", "scale=950:782",
            "-c:v", "libx264",
            "-pix_fmt", "yuv420p",
            "-b:v", "750k",
            f"month_{month}.mp4"
        ]
        subprocess.run(cmd)

Generating video for Jan


ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.3.0 (conda-forge gcc 12.3.0-3)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1703992514465/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1703992514465/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1703992514465/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1703992514465/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1703992514465/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libo

Generating video for Feb


[libx264 @ 0x563d111af7c0] using SAR=2970/3077
[libx264 @ 0x563d111af7c0] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2 AVX512
[libx264 @ 0x563d111af7c0] profile High, level 3.1, 4:2:0, 8-bit
[libx264 @ 0x563d111af7c0] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=-2 threads=24 lookahead_threads=4 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=25 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=abr mbtree=1 bitrate=750 ratetol=1.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to 'month_Feb.mp4':
  Metadata:
    encoder         : Lavf60.1

Generating video for Mar


...

ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.3.0 (conda-forge gcc 12.3.0-3)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1703992514465/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1703992514465/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1703992514465/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1703992514465/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1703992514465/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libo

Generating video for Nov


[libx264 @ 0x55f5af21f3c0] using SAR=2970/3077
[libx264 @ 0x55f5af21f3c0] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2 AVX512
[libx264 @ 0x55f5af21f3c0] profile High, level 3.1, 4:2:0, 8-bit
[libx264 @ 0x55f5af21f3c0] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=-2 threads=24 lookahead_threads=4 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=25 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=abr mbtree=1 bitrate=750 ratetol=1.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to 'month_Nov.mp4':
  Metadata:
    encoder         : Lavf60.1

Generating video for Dec


[libx264 @ 0x55e9277f1e80] using SAR=2970/3077
[libx264 @ 0x55e9277f1e80] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2 AVX512
[libx264 @ 0x55e9277f1e80] profile High, level 3.1, 4:2:0, 8-bit
[libx264 @ 0x55e9277f1e80] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=-2 threads=24 lookahead_threads=4 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=25 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=abr mbtree=1 bitrate=750 ratetol=1.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to 'month_Dec.mp4':
  Metadata:
    encoder         : Lavf60.1

In [7]:
with open("concat_list.txt", "w") as f:
    for month in MONTH_NAMES:
        f.write(f"file 'month_{month}.mp4'\n")


**Make Full Year Video!**

In [8]:
concat_cmd = [
    "ffmpeg", "-y", "-f", "concat", "-safe", "0",
    "-i", "concat_list.txt",
    "-c", "copy",
    "full_year_video.mp4"
]

subprocess.run(concat_cmd)
print("Done!")


ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.3.0 (conda-forge gcc 12.3.0-3)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1703992514465/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1703992514465/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1703992514465/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1703992514465/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1703992514465/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libo

Done!


[mov,mp4,m4a,3gp,3g2,mj2 @ 0x7f04f40e6100] Auto-inserting h264_mp4toannexb bitstream filter
[out#0/mp4 @ 0x56239db87240] video:19553kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.214626%
size=   19595kB time=00:03:43.12 bitrate= 719.4kbits/s speed= 234x    


---

# Generate for a Single Day

In [9]:
def render_day(day, hours=range(0, 24, 3)):
    output_dir = f"frames_test_day_{day}"
    os.makedirs(output_dir, exist_ok=True)
    
    for hour in hours:
        lat, lon, decl = get_subsolar_point(day, hour)
        cos_zenith = calculate_zenith_cosine(decl, hour)
        
        projections = [
            ccrs.AzimuthalEquidistant(central_longitude=0, central_latitude=90),
            ccrs.AzimuthalEquidistant(central_longitude=0, central_latitude=-90),
            ccrs.Mollweide(central_longitude=0)
        ]
        
        fig = plt.figure(figsize=(12, 8))
        gs = GridSpec(2, 2, height_ratios=[1.2, 1])
        
        ax1 = plt.subplot(gs[0, 0], projection=projections[0])
        ax2 = plt.subplot(gs[0, 1], projection=projections[1])
        ax3 = plt.subplot(gs[1, :], projection=projections[2])
        
        axes = [ax1, ax2, ax3]
        
        for i, ax in enumerate(axes):
            ax.set_global()
            ax.coastlines()
            ax.add_feature(cfeature.BORDERS, linewidth=0.5)
            ax.gridlines(draw_labels=False, linewidth=0.3, linestyle='--')
            mesh = ax.pcolormesh(
                LONS, LATS, SolarConstant*cos_zenith, transform=ccrs.PlateCarree(),
                cmap='inferno', shading='auto', vmin=0, vmax=1
            )
            ax.plot(lon, lat, 'o', color='yellow', transform=ccrs.PlateCarree(), zorder=5)
            ax.set_title(f"{titles_text[i]}", fontsize=10)
        
        # Adjust layout and add colorbar & title as before
        fig.subplots_adjust(top=0.88, bottom=0.12)
        cb_ax = fig.add_axes([0.125, 0.08, 0.75, 0.03])
        cb = plt.colorbar(mesh, cax=cb_ax, orientation='horizontal')
        ticks = [0, 200, 400, 600, 800, 1000, 1200]
        cb.set_ticks(ticks)
        cb.set_label("Estimated Solar Irradiance (W/m$^{2}$)")

        if day in events:
            fig.suptitle(f"Sunlight Distribution on Day {day} — {hour:02d}:00 UTC\n" + events[day],
                fontsize=14, ha='center', y=.95)
        else:
            fig.suptitle(f"Sunlight Distribution on Day {day} — {hour:02d}:00 UTC",
                         fontsize=14, ha='center', y=.95)
            
        
        filename = os.path.join(output_dir, f"frame_{day:03d}_{hour:02d}.png")
        fig.savefig(filename, dpi=100, bbox_inches='tight')
        plt.close(fig)
    print(f"Rendered day {day} frames in {output_dir}")